In [1]:
from pathlib import Path

import re

from collections import defaultdict, deque

In [2]:
data_path = Path.home() / 'workstation' / 'dev' / 'Advent-of-Code-2020' / 'data' / 'day7_input.txt'

In [3]:
data_path.exists()

True

In [4]:
with open(data_path, 'r') as reader:
    text_input = reader.read().strip()

In [5]:
bag_hierarchies = text_input.split('\n')

In [6]:
regex_pattern = re.compile(r'\w+\s\w+(?=\sbag)')

# dictionary of bags with the 'values' being the bag types that can be contained in (are smaller than) the 'key'
bag_dict = defaultdict(list)

for hierarchy in bag_hierarchies:
    bag_list = regex_pattern.findall(hierarchy)
    if 'no other' in bag_list:
        # Smallest bags. Nothing smaller than them
        bag_list[1] = 'None'
    
    # This is done so that if the same 'key' appears, we append to the existing list, rather than re-write it
    for bag_type in bag_list[1:]:
        bag_dict[bag_list[0]].append(bag_type)

In [7]:
# In this dict, now 'values' for a particular 'key' signify bags than can contain (are larger than) the 'key'

inverted_bag_dict = defaultdict(list)

for key, value in bag_dict.items():
    for bag_type in value:
        inverted_bag_dict[bag_type].append(key)

In [8]:
len(bag_dict.keys())

594

In [9]:
# Some bags are not contained in any other bags, i.e., the largest bags. Not mentioned explicitly

len(inverted_bag_dict.keys())

421

In [10]:
# Identifying the largest bags

bags_not_contained_anywhere = [key for key in list(bag_dict.keys()) if key not in list(inverted_bag_dict.keys())]

In [11]:
bags_not_contained_anywhere[:3]

['dotted crimson', 'dark lime', 'vibrant turquoise']

In [12]:
for bag in bags_not_contained_anywhere:
    inverted_bag_dict[bag].append('None')

In [13]:
# Node: A certain bag type
# Child node: A bag type that is larger than the parent node
# BFS employed here. Probably unncessary

larger_bags = set()
queue = deque()
queue.append('shiny gold')

while queue:
    level_size = len(queue)
    current_level = []
    for _ in range(level_size):
        current_node = queue.popleft()
        # add the node to the current level
        current_level.append(current_node)
        # insert the children of current node in the queue, if children exist
        list_children_nodes = inverted_bag_dict[current_node]
        if list_children_nodes != ['None']:
            queue.extend(list_children_nodes)
    larger_bags.update(current_level)

In [14]:
larger_bags.remove('shiny gold')

In [15]:
len(larger_bags)

337

#### Part 2

In [16]:
# Using string.split() to separate the bags based on containment
bag_hierarchies[1].split(' contain ')

['mirrored brown bags',
 '1 pale teal bag, 3 muted gray bags, 3 dark bronze bags.']

In [17]:
regex_pattern = re.compile(r'\w+\s\w+(?=\sbag)')
regex_pattern_2 = re.compile(r'\d+\s\w+\s\w+(?=\sbag)')

# dictionary of bags with the 'values' being the bag types that can be contained in (are smaller than) the 'key'
bag_dict = defaultdict(list)

for hierarchy in bag_hierarchies:
    bag_list = hierarchy.split(' contain ')
    dict_key = regex_pattern.search(bag_list[0]).group()

    smaller_bag_types = regex_pattern_2.findall(bag_list[1])
    # Smallest bags
    if not smaller_bag_types:
        bag_dict[dict_key].append('None')
    else:
        for bag_type in smaller_bag_types:
            bag_dict[dict_key].append(bag_type)

In [18]:
def count_num_bags(dict_key):
    count = 0
    
    if bag_dict[dict_key] == ['None']:
        return count
    
    for bag in bag_dict[dict_key]:
        bag_count, bag_type = bag.split(' ', maxsplit=1)
        bag_count = int(bag_count)
        count += bag_count
        count += bag_count*count_num_bags(bag_type)
    
    return count

In [19]:
count_num_bags('shiny gold')

50100